# Notes
/!\ This notebook is fully functional on Google Colab./!\
At the beginning of each section, we've added a small reminder explaining the algorithm that we're being asked to implement.

# Reinforcement Learning with Function Approximation

In [0]:
!pip install torch==1.2.0 torchvision pyvirtualdisplay matplotlib seaborn pandas numpy pathlib gym
!sudo apt-get install xvfb
!git clone https://github.com/rlgammazero/mvarl_hands_on.git > /dev/null 2>&1
!cd mvarl_hands_on && git pull origin master > /dev/null 2>&1

In [0]:
import itertools

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch import optim

import numpy as np

import pandas as pd

import gym
from gym.wrappers import Monitor
from pprint import pprint
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from IPython.display import clear_output

# The following code is will be used to visualize the environments.

def show_video(directory):
    html = []
    for mp4 in Path(directory).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
    
display = Display(visible=0, size=(1400, 900))
display.start();

def make_seed(seed):
    np.random.seed(seed=seed)
    torch.manual_seed(seed=seed)

In [0]:
!apt-get install x11-utils 
from pathlib import Path
import base64

PyTorch is a Python package that provides two high-level features:
- Tensor computation (like NumPy) with strong GPU acceleration
- Deep neural networks built on a tape-based autograd system

At a granular level, PyTorch is a library that consists of the following components:

| Component | Description |
| ---- | --- |
| [**torch**](https://pytorch.org/docs/stable/torch.html) | a Tensor library like NumPy, with strong GPU support |
| [**torch.autograd**](https://pytorch.org/docs/stable/autograd.html) | a tape-based automatic differentiation library that supports all differentiable Tensor operations in torch |
| [**torch.jit**](https://pytorch.org/docs/stable/jit.html) | a compilation stack (TorchScript) to create serializable and optimizable models from PyTorch code  |
| [**torch.nn**](https://pytorch.org/docs/stable/nn.html) | a neural networks library deeply integrated with autograd designed for maximum flexibility |
| [**torch.multiprocessing**](https://pytorch.org/docs/stable/multiprocessing.html) | Python multiprocessing, but with magical memory sharing of torch Tensors across processes. Useful for data loading and Hogwild training |
| [**torch.utils**](https://pytorch.org/docs/stable/data.html) | DataLoader and other utility functions for convenience |


**Tutorials on PyTorch:** https://pytorch.org/tutorials/


## OpenAI gym
We will consider environments provided by OpenAI gym
This library provides a large number of environments to test RL algorithm.

We will focus on the **CartPole-v1** environment in this lab but we encourage you to also test your code on:
* **Acrobot-v1**
* **MountainCar-v0**

| Env Info          	| CartPole-v1 	| Acrobot-v1                	| MountainCar-v0 	|
|-------------------	|-------------	|---------------------------	|----------------	|
| **Observation Space** 	| Box(4)      	| Box(6)                    	| Box(2)         	|
| **Action Space**      	| Discrete(2) 	| Discrete(3)               	| Discrete(3)    	|
| **Rewards**           	| 1 per step  	| -1 if not terminal else 0 	| -1 per step    	|

A gym environment is loaded with the command `env = gym.make(env_id)`. Once the environment is created, you need to reset it with `observation = env.reset()` and then you can interact with it using the method step: `observation, reward, done, info = env.step(action)`.

In [0]:
# We load CartPole-v1
env = gym.make('CartPole-v1')
# We wrap it in order to save our experiment on a file.
env = Monitor(env, "./gym-results", force=True)

In [0]:
done = False
obs = env.reset()
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
env.close()
show_video("./gym-results")

## REINFORCE

**Q1: Implement the REINFORCE algorithm**

The code is splitted in two parts:
* The Model class defines the architecture of our neural network which takes as input the current state and returns the policy,
* The Agent class is responsible for the training and evaluation procedure. You will need to code the method `optimize_model`.

In [0]:
class Model(nn.Module):
    def __init__(self, dim_observation, n_actions):
        super(Model, self).__init__()
        
        self.n_actions = n_actions
        self.dim_observation = dim_observation
        
        self.net = nn.Sequential(
            nn.Linear(in_features=self.dim_observation, out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=8),
            nn.ReLU(),
            nn.Linear(in_features=8, out_features=self.n_actions),
            nn.Softmax(dim=0)
        )
        
    def forward(self, state):
        return self.net(state)
    
    def select_action(self, state):
        action = torch.multinomial(self.forward(state), 1)
        return action

Create the model based on the properties of the MDP you want to solve

In [0]:
env_id = 'CartPole-v1'
env = gym.make(env_id)
model = Model(env.observation_space.shape[0], env.action_space.n)
print(f'The model we created correspond to:\n{model}')

We provide a base agent that you will need to extend in the next cell with your implementation of `optimize_model`.

In [0]:
import pandas as pd
import itertools
import seaborn as sns

class BaseAgent:
    
    def __init__(self, config):
        self.config = config
        self.env = gym.make(config['env_id'])
        make_seed(config['seed'])
        self.env.seed(config['seed'])
        self.model = Model(self.env.observation_space.shape[0], self.env.action_space.n)
        self.gamma = config['gamma']
        
        # the optimizer used by PyTorch (Stochastic Gradient, Adagrad, Adam, etc.)
        self.optimizer = torch.optim.Adam(self.model.net.parameters(), lr=config['learning_rate'])
        self.monitor_env = Monitor(env, "./gym-results", force=True, video_callable=lambda episode: True)
    
    # Method to implement
    def _compute_returns(self, rewards):
        """Returns the cumulative discounted rewards at each time step

        Parameters
        ----------
        rewards : array
            The array of rewards of one episode

        Returns
        -------
        array
            The cumulative discounted rewards at each time step
            
        Example
        -------
        for rewards=[1, 2, 3] this method outputs [1 + 2 * gamma + 3 * gamma**2, 2 + 3 * gamma, 3] 
        """
        
        raise NotImplementedError
    
    # Method to implement
    def optimize_model(self, n_trajectories):
        """Perform a gradient update using n_trajectories

        Parameters
        ----------
        n_trajectories : int
            The number of trajectories used to approximate the expectation card(D) in the formula above
        
        Returns
        -------
        array
            The cumulative discounted rewards of each trajectory
        """
        
        raise NotImplementedError
    
    def train(self, n_trajectories, n_update):
        """Training method

        Parameters
        ----------
        n_trajectories : int
            The number of trajectories used to approximate the expected gradient
        n_update : int
            The number of gradient updates
            
        """
        
        rewards = []
        for episode in range(n_update):
            rewards.append(self.optimize_model(n_trajectories))
            print(f'Episode {episode + 1}/{n_update}: rewards {round(rewards[-1].mean(), 2)} +/- {round(rewards[-1].std(), 2)}')
        # Plotting
        r = pd.DataFrame((itertools.chain(*(itertools.product([i], rewards[i]) for i in range(len(rewards))))), columns=['Epoch', 'Reward'])
        sns.lineplot(x="Epoch", y="Reward", data=r, ci='sd');
        
    def evaluate(self, render=False):
        """Evaluate the agent on a single trajectory            
        """
        with torch.no_grad():
            observation = self.monitor_env.reset()
            observation = torch.tensor(observation, dtype=torch.float)
            reward_episode = 0
            done = False
                
            while not done:
                action = self.model.select_action(observation)
                observation, reward, done, info = self.monitor_env.step(int(action))
                observation = torch.tensor(observation, dtype=torch.float)
                reward_episode += reward
            
            self.monitor_env.close()
            if render:
                show_video("./gym-results")
                print(f'Reward: {reward_episode}')
            print(f'Reward: {reward_episode}')

Finally you can implement your agent

####**Solution**

We approximate the policy $\pi_\theta$ (where $\theta$ parametrizes the neural network defined in the *Model* class) using the REINFORCE (with temporal structure) algorithm. We recall that we're interested in finding the $\theta^\star$ which maximizes the criterion
$$
J(\pi_\theta) = \mathbb{E}_{}\left[ \sum_{t} \gamma^{t} r_t \Big| a \sim \pi_\theta \right].
$$
We want to use a *gradient ascent* method. The gradient of $J$ is given by
$$
\nabla_\theta J(\pi_\theta) = \mathbb{E}\left[ \sum_t \gamma^t r_t \sum_t \nabla_\theta \log \pi_\theta(s_t, a_t) \Big| a \sim \pi_\theta\right],
$$
and therefore we introduce the loss (with temporal structure to reduce the variance)
$$
L(\theta) = \frac{1}{n} \sum_{i = 1}^n \left( \sum_{t = 1}^{T^i} \log\pi_\theta(s_t^i, a_t^i) \sum_{t' = t}^{T^i} \gamma^{t - t'}r_{t'}^i \right),
$$
where $n$ is the number of trajectories and $T^i$ some stopping times.



In [0]:
class REINFORCE(BaseAgent):
    
    def _compute_returns(self, rewards):
        s = len(rewards)
        d = [self.gamma**i for i in range(s)]
        return [np.dot(d[:s - i], rewards[i:]) for i in range(s)]
    
        
    def optimize_model(self, n_trajectories):
        reward_trajectories = np.empty(n_trajectories)
        loss = 0.
        
        for i in range(n_trajectories):
          
          rewards = []
          probas = []

          done = False
          obs = torch.from_numpy(self.env.reset()).float()
          while not done:
            action = self.model.select_action(obs)
            probas.append(self.model(obs)[int(action)])
            obs, reward, done, info = self.env.step(int(action))
            obs = torch.from_numpy(obs).float()
            rewards.append(reward)

          probas = torch.stack(probas)
          
          
          log_probas = torch.log(probas).float()
          
          disc_rewards = np.array(self._compute_returns(rewards))
          disc_rewards = torch.from_numpy(disc_rewards).float()
          loss = loss + torch.dot(log_probas, disc_rewards)/n_trajectories
            

          reward_trajectories[i] = disc_rewards.numpy()[0]
        
        
        env.close()

        loss = -loss

        # The following lines take care of the gradient descent step for the variable loss
        # that you need to compute.
        
        # Discard previous gradients
        self.optimizer.zero_grad()
        # Compute the gradient 
        loss.backward()
        # Do the gradient descent step
        self.optimizer.step()
        return reward_trajectories
    

In [0]:
env_id = 'CartPole-v1'
learning_rate = 0.01
gamma = 1 
seed = 1235

config = {
    'env_id': env_id,
    'learning_rate': learning_rate,
    'seed': seed,
    'gamma': gamma
}

print("Current config is:")
pprint(config)


In [0]:
agent = REINFORCE(config)
agent.train(n_trajectories=50, n_update=50)

Evaluate the agent over multiple episodes

In [0]:
agent.evaluate(render = True)

## Policy Evaluation as Supervised Learning

**Q2: Implement batched gradient algorithm**

Define network for Q-function (ValueNetwork) and policy (ActorNetwork)


### **Solution**

Given a policy $\pi$ (given by the neural network *ActorNetwork*), the goal is to approximate the associated value-function $V^{\pi}$ with a parametric model $V^{\pi}_\theta$ (given by the neural network *ValueNetwork*). We train the model using *batched gradient descent*: that is, we generate a trajectory 
$$
(s_1, a_1, s_2, a_2, \dots, s_m, a_m),
$$
with $m$ the batch size and $T_i$'s are the stopping times (i.e. $T_i$ is the index at which terminates the chain to which $s_i$ belongs, *e.g.* if we observe the trajectory $(s_1, s_2, s_3, s_4)$ with $s_4$ terminal, then $T_1 = T_2 = T_3 = T_4 = 4$), and define the loss on that very trajectory as 
$$
L(\theta) = \sum_{i = 1}^m \left(V^\pi_\theta(s_i) - \sum_{t = i}^{T_i} \gamma^{t - i}r_t\right)^2.
$$
***Remark:*** In pratice, for $s_m$ is not necessarily a terminal state, the reward on the unfinished trajectory (starting say at index $k$) is given by $\sum_{t = k}^m \gamma^{t - k}r_t + \gamma V^\pi_{\theta^{(c)}}(s_m)$ where $\theta^{(c)}$ is the current parametrization of *ValueNetwork* (*i.e.* **bootstrapping**).




In [0]:
 class ValueNetwork(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(ValueNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
    
    def predict(self, x):
        return self(x).detach().numpy()[0]

In [0]:
class ActorNetwork(nn.Module):

    def __init__(self, input_size, hidden_size, action_size):
        super(ActorNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        out = F.softmax(self.fc3(out), dim=-1)
        return out
    
    def select_action(self, x):
        return torch.multinomial(self(x), 1).detach().numpy()

Implement your (batched) gradient algorithm



In [0]:
class EvalAgent:

    def __init__(self, config, policy):
        self.config = config
        self.env = gym.make(config['env_id'])
        make_seed(config['seed'])
        self.env.seed(config['seed'])
        self.monitor_env = Monitor(self.env, "./gym-results", force=True, video_callable=lambda episode: True)
        self.gamma = config['gamma']
        self.policy = policy

        # Our network
        self.value_network = ValueNetwork(self.env.observation_space.shape[0], 16, 1)

        # optimizers
        self.value_network_optimizer = optim.RMSprop(self.value_network.parameters(), 
                                                   lr=config['value_network']['learning_rate'])
    
    def _returns_advantages(self, rewards, dones, next_value):
        """Returns the cumulative discounted rewards at each time step

        Parameters
        ----------
        rewards : array
            An array of shape (batch_size,) containing the rewards given by the env
        dones : array
            An array of shape (batch_size,) containing the done bool indicator given by the env
        next_value : float
            The value of the next state given by the value network

        Returns
        -------
        returns : array
            The cumulative discounted rewards
        """
        returns = np.zeros(len(rewards))
        for i in range(1, len(rewards) + 1):
            next_value = rewards[-i] + self.gamma * int(not dones[-i]) * next_value
            returns[-i] = next_value

        return returns
    
    
    def optimize_model(self, observations, returns):
        """Perform a gradient update using provided transitions

        Parameters
        ----------
        observations : array
            The observations
        actions : array
            The actions
        returns : array
            The returns from each state

        Returns
        -------
        loss_value: int
            The loss value
        """

        returns = torch.tensor(returns[:, None], dtype=torch.float)
        observations = torch.tensor(observations, dtype=torch.float)

        values = self.value_network(observations) 
        loss = F.mse_loss(values, returns)
        
        loss.backward()
        self.value_network_optimizer.step() 
        self.value_network_optimizer.zero_grad()


    
    def training_batch(self, epochs, batch_size):
        """Perform a training by batch

        Parameters
        ----------
        epochs : int
            Number of epochs
        batch_size : int
            The size of a batch
        """
        episode_count = 0
        actions = np.empty((batch_size,), dtype=np.int)
        dones = np.empty((batch_size,), dtype=np.bool)
        rewards = np.empty((batch_size, ), dtype=np.float)
        observations = np.empty((batch_size,) + self.env.observation_space.shape, dtype=np.float)
        observation = self.env.reset()
        mse_test = []


        for epoch in range(epochs):
            # Lets collect one batch
            for i in range(batch_size):
                observations[i] = observation
                observation_t = torch.from_numpy(observation).float()
                actions[i] = int(self.policy.select_action(observation_t))
                observation, rewards[i], dones[i], _ = self.env.step(actions[i])

                if dones[i]:
                    observation = self.env.reset()

            # If our episode didn't end on the last step we need to compute the value for the last state
            if dones[-1]:
                next_value = 0
            else:
                next_value = self.value_network(torch.from_numpy(observation).float())

            # Update episode_count
            episode_count += sum(dones)

            # Compute returns
            returns = self._returns_advantages(rewards, dones, next_value)

            # Learning step !
            self.optimize_model(observations, returns)

            # Test it every 50 epochs
            if epoch % 25 == 0 or epoch == epochs - 1:
                L = []
                for _ in range(10):
                    obs_states, y_mc = self.evaluate()
                    obs_states = torch.from_numpy(obs_states).float()
                    y_hat = self.value_network(obs_states).detach().numpy()
                    err = y_mc - y_hat
                    mse = np.mean(err**2)
                    L.append(mse.item())
                mse_test.append(L)
                print(f'Epoch {epoch}/{epochs}: MSE: {np.mean(mse)}')

                observation = self.env.reset()

        # Plotting
        r = pd.DataFrame((itertools.chain(*(itertools.product([i], mse_test[i]) for i in range(len(mse_test))))), columns=['Epoch', 'MSE'])
        sns.lineplot(x="Epoch", y="MSE", data=r, ci='sd');

        print(f'The trainnig was done over a total of {episode_count} episodes')

    def evaluate(self, render=False):
        """Returns the observations and the estimated V-function (using first visit Monte-Carlo)

        Parameters
        ----------
        rewards : array
            An array of shape (batch_size,) containing the rewards given by the env
        dones : array
            An array of shape (batch_size,) containing the done bool indicator given by the env
        values : array
            An array of shape (batch_size,) containing the values given by the value network
        next_value : float
            The value of the next state given by the value network

        Returns
        -------
        states : array
            Observations
        returns : array
            The estimate value function of each state
        """
        
        env = self.monitor_env if render else self.env
        observation = env.reset()
        states = [observation.copy()]
        rewards= []
        observation = torch.tensor(observation, dtype=torch.float)
        done = False
        steps = 0

        while not done:
            action = self.policy.select_action(observation)
            observation, reward, done, info = env.step(int(action))
            for i in range(steps):
                rewards[i] = rewards[i] + math.pow(self.gamma, steps-i)*reward
            rewards.append(reward)
            if not done:
                states.append(observation.copy())
            observation = torch.tensor(observation, dtype=torch.float)
            steps += 1

        env.close()
        if render:
            show_video("./gym-results")
            print(f'Reward: {reward_episode}')
        states = np.array(states).reshape(-1, self.env.observation_space.shape[0])
        returns = np.array(rewards).reshape(-1,1)
        return states, returns

Define configuration

In [0]:
env_id = 'CartPole-v1'
value_learning_rate = 0.001
gamma = 0.99
seed = 1

config_td = {
    'env_id': env_id,
    'gamma': gamma,
    'seed': seed,
    'value_network': {'learning_rate': value_learning_rate, 'reference': './CartPole_value.pt'}
}

print("Current config_td is:")
pprint(config_td)


Create policy

In [0]:
env = gym.make(config_td['env_id'])
policy = ActorNetwork(env.observation_space.shape[0], 16, env.action_space.n)
policy.load_state_dict(torch.load('./mvarl_hands_on/data/CartPole_actor.pt'))
state = torch.tensor(np.array(env.reset(), dtype=np.float32))
print("pi(state) = ", policy.select_action(state))
del env

Run agent

In [0]:
import math
agent = EvalAgent(config=config_td, policy=policy)
agent.training_batch(epochs=1000, batch_size=256)

## Advantage Actor Critic (A2C)
**Q3: Implement the A2C method**

As usual we provide a structure you can use as starting point.



**Note:** try to reuse previous parts of previous code.



### Solution

***Reminder*** : Recall that in the REINFORCE algorithm (with temporal structure), the optimal policy $\pi^\star$ is sought after using a parametric model $\pi_\theta$ (here, the neural network *ActorNetwork*). The parametric model is trained by maximizing the criterion
$$
L(\theta) = \frac{1}{n} \sum_{i = 1}^n \left( \sum_{t = 1}^{T^i} \log\pi_\theta(s_t^i, a_t^i) \sum_{t' = t}^{T^i} \gamma^{t - t'}r_{t'}^i \right),
$$
where we sampled a trajectory $n$ trajectories $(s^i_1, a^i_1, s^i_2, a^i_2\dots, s^i_{T^i}, a^i_{T^i})$ with stopping times $T^i$'s. This criterion is an *ansatz* so that its gradient constitutes a consistent Monte-Carlo approximation of the true gradient of $J$, *i.e.*
$$
\nabla_\theta J(\pi_\theta) = \mathbb{E}\left[ \sum_{t = 1}^T \nabla_\theta \log \pi_\theta(s_t, a_t) \sum_{t' = t}^T \gamma^{t' - t} r_t  \Big| a \sim \pi_\theta\right].
$$
***Baseline*** : As usual, this MC estimator might have high variance. To tackle this issue, one way to go in the famous *control variates method*, in which we introduce an unbiased variable $b$ (here called the *baseline*) to reduce the variance without affecting the consistence of the estimator:

$$
\nabla_\theta J(\pi_\theta) = \mathbb{E}\left[ \sum_{t = 1}^T \nabla_\theta \log \pi_\theta(s_t, a_t) \left(\sum_{t' = t}^T \gamma^{t' - t} r_t - b(s_t)\right)  \Big| a \sim \pi_\theta\right].
$$

***A2C***: In the *Advantage Actor-Critic* approach, two parametric models $\theta \to \pi_\theta$ (the **actor**, here the neural network *ActionNetwork*) and $\pi_\theta \to V^{\pi_\theta}$ (the **critic**, here the neural network *ValueNetwork*) are going to work with each other to improve their respective performances. We have already implemented the entire pipeline for the critic in the previous exercise. Now, what about the actor? Well, it can be shown  that the optimal choice for the base $b$ is nothing but that the value function $V^\pi$ itself. The idea is therefore to do some bootstrapping, that is given a current parametrization $\theta^{(c)}$ for the critic, we want to train the actor $\theta \to \pi_\theta$ by maximizing the criterion
$$
L(\theta) = \frac{1}{n} \sum_{i = 1}^n \Bigg( \sum_{t = 1}^{T^i} \log\pi_\theta(s_t^i, a_t^i) \underbrace{\left( \sum_{t' = t}^{T^i} \gamma^{t - t'}r_{t'}^i - V_{\theta^{(c)}}(s_{t})\right)}_{\textrm{ << Advantage >>}} \Bigg)
$$

***Entropy***: In the following implementation of A2C, we add to $L(\theta)$ the entropy $\textrm{Ent}(\pi_\theta) = - \sum_{s,a} \pi_\theta(s,a) \log(\pi_\theta(s,a))$. This forces the entropy of $\pi_\theta$ to get higher, therefore flattening the distribution, which in turn enforces exploration. 

In [0]:
class A2CAgent:

    def __init__(self, config):
        self.config = config
        self.env = gym.make(config['env_id'])
        make_seed(config['seed'])
        self.env.seed(config['seed'])
        self.monitor_env = Monitor(self.env, "./gym-results", force=True, video_callable=lambda episode: True)
        self.gamma = config['gamma']
        self.entropy = config['entropy']
        # Our two networks
        self.value_network = ValueNetwork(self.env.observation_space.shape[0], 16, 1)
        self.actor_network = ActorNetwork(self.env.observation_space.shape[0], 16, self.env.action_space.n)
        
        # Their optimizers
        self.value_network_optimizer = optim.RMSprop(self.value_network.parameters(), lr=config['value_network']['learning_rate'])
        self.actor_network_optimizer = optim.RMSprop(self.actor_network.parameters(), lr=config['actor_network']['learning_rate'])
        
    # Hint: use it during training_batch
    def _returns_advantages(self, rewards, dones, values, next_value):
        """Returns the cumulative discounted rewards at each time step

        Parameters
        ----------
        rewards : array
            An array of shape (batch_size,) containing the rewards given by the env
        dones : array
            An array of shape (batch_size,) containing the done bool indicator given by the env
        values : array
            An array of shape (batch_size,) containing the values given by the value network
        next_value : float
            The value of the next state given by the value network
        
        Returns
        -------
        returns : array
            The cumulative discounted rewards
        advantages : array
            The advantages
        """
        returns = np.zeros(len(rewards))
        for i in range(1, len(rewards) + 1):
            next_value = rewards[-i] + self.gamma * int(not dones[-i]) * next_value
            returns[-i] = next_value

        return returns, returns - values 

    def training_batch(self, epochs, batch_size):
        """Perform a training by batch

        Parameters
        ----------
        epochs : int
            Number of epochs
        batch_size : int
            The size of a batch
        """
        episode_count = 0
        actions = np.empty((batch_size,), dtype=np.int)
        dones = np.empty((batch_size,), dtype=np.bool)
        rewards, values = np.empty((2, batch_size), dtype=np.float)
        observations = np.empty((batch_size,) + self.env.observation_space.shape, dtype=np.float)
        observation = self.env.reset()
        rewards_test = []

        for epoch in range(epochs):

            # Lets collect one batch

            for i in range(batch_size):

                observations[i] = observation
                observation_t = torch.from_numpy(observation).float()
                actions[i] = int(self.actor_network.select_action(observation_t))
                values[i] = self.value_network(observation_t)
                observation, rewards[i], dones[i], _ = self.env.step(actions[i])

                if dones[i]:
                    observation = self.env.reset()

            # If our episode didn't end on the last step we need to compute the value for the last state
            if dones[-1]:
                next_value = 0
            else:
                next_value = self.value_network(torch.from_numpy(observation).float())

            # Update episode_count
            episode_count += sum(dones)

            # Compute returns and advantages
            returns, advantages = self._returns_advantages(rewards, dones, values, next_value)
           
            # Learning step !
            self.optimize_model(observations, actions, returns, advantages)

            # Test it every 50 epochs
            if epoch % 50 == 0 or epoch == epochs - 1:
                rewards_test.append(np.array([self.evaluate() for _ in range(50)]))
                print(f'Epoch {epoch}/{epochs}: Mean rewards: {round(rewards_test[-1].mean(), 2)}, Std: {round(rewards_test[-1].std(), 2)}')

                # Early stopping
                if rewards_test[-1].mean() > 490 and epoch != epochs -1:
                    print('Early stopping !')
                    break
                observation = self.env.reset()
                    
        # Plotting
        r = pd.DataFrame((itertools.chain(*(itertools.product([i], rewards_test[i]) for i in range(len(rewards_test))))), columns=['Epoch', 'Reward'])
        sns.lineplot(x="Epoch", y="Reward", data=r, ci='sd');
        
        print(f'The trainnig was done over a total of {episode_count} episodes')

    def optimize_model(self, observations, actions, returns, advantages):
        returns = torch.tensor(returns[:, None], dtype=torch.float)
        advantages = torch.tensor(advantages, dtype=torch.float)
        observations = torch.tensor(observations, dtype=torch.float)

        # MSE for the values

        returns = torch.tensor(returns[:, None], dtype=torch.float)
        observations = torch.tensor(observations, dtype=torch.float)

        values = self.value_network(observations) 
        loss = F.mse_loss(values, returns)
        
        loss.backward()
        self.value_network_optimizer.step() 
        self.value_network_optimizer.zero_grad()


        # Actor & Entropy loss

        values = self.actor_network(observations)
        loss = 0.
        for i in range(len(observations)):
            loss += torch.log(values[i, actions[i]])*advantages[i]
        
        loss = -loss + self.entropy * torch.sum(values * torch.log(values))
        loss.backward()
        self.actor_network_optimizer.step()
        self.actor_network_optimizer.zero_grad()


    def evaluate(self, render=False):
        env = self.monitor_env if render else self.env
        observation = env.reset()
        observation = torch.tensor(observation, dtype=torch.float)
        reward_episode = 0
        done = False

        while not done:
            policy = self.actor_network(observation)
            action = torch.multinomial(policy, 1)
            observation, reward, done, info = env.step(int(action))
            observation = torch.tensor(observation, dtype=torch.float)
            reward_episode += reward
            
        env.close()
        if render:
            show_video("./gym-results")
            print(f'Reward: {reward_episode}')
        return reward_episode

Create configuration for A2C

In [0]:
env_id = 'CartPole-v1'
value_learning_rate = 0.001
actor_learning_rate = 0.001
gamma = 0.99
entropy = 1
seed = 1

config_a2c = {
    'env_id': env_id,
    'gamma': gamma,
    'seed': seed,
    'value_network': {'learning_rate': value_learning_rate},
    'actor_network': {'learning_rate': actor_learning_rate},
    'entropy': entropy
}

print("Current config_a2c is:")
pprint(config_a2c)

Run the learning agent

In [0]:
agent = A2CAgent(config_a2c)
rewards = agent.training_batch(1000, 256)

Evaluate the agent over multiple episodes

In [0]:
agent.evaluate(render = True)